In [1]:
#@title <b>💻 Run interactive application</b> { display-mode: "form" }

!git clone https://github.com/Tsuchihashi-ryo/proteovis.git
%cd proteovis
!pip install .
from IPython.display import clear_output
clear_output()

import matplotlib.pyplot as plt
import proteovis as pv
import pandas as pd
import ipywidgets as widgets
import io
import contextlib
import warnings
from IPython.display import display, clear_output

upload = widgets.FileUpload(accept='.zip,.res,.result', multiple=False)

display(widgets.HTML("<h3>Upload a .zip, .res or .result file:</h3>"))
display(upload)

def slider_text_pair(description, init, minv, maxv, step=1.0):
    slider = widgets.FloatSlider(value=init, min=minv, max=maxv, step=step, description=description)
    text = widgets.FloatText(value=init, layout=widgets.Layout(width="100px"))
    widgets.jslink((slider, 'value'), (text, 'value'))
    return widgets.HBox([slider, text]), slider

def plot_chromatogram(df, data, xmin, xmax, ymin, ymax,
                      y2_label, y2_ymin, y2_ymax,
                      show_fractions, tick_h, min_spacing, font_frac,
                      font_title, font_labels, font_ticks, font_legend,
                      nom_fitxer):

    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax1.plot(df["mL"], df["UV 1_280"], label="UV 1_280", color="blue")
    ax1.plot(df["mL"], df["UV 2_260"], label="UV 2_260", color="red")
    ax1.set_xlim(xmin, xmax)
    ax1.set_ylim(ymin, ymax)
    ax1.set_xlabel("Elution volume (mL)", fontsize=font_labels)
    ax1.set_ylabel("Absorbance (mAU)", fontsize=font_labels)
    ax1.tick_params(axis='both', labelsize=font_ticks)
    ax1.grid(True)

    if show_fractions:
        last_label_x = None
        fractions = df[(df['Fractions'].notna()) & (df['mL'].between(xmin, xmax))].reset_index()
        for i in range(len(fractions)):
            x = fractions.loc[i, 'mL']
            ax1.vlines(x, ymin, ymin + tick_h, color='red', linewidth=1)
            if i % 2 == 0 and (last_label_x is None or abs(x - last_label_x) > min_spacing):
                label = fractions.loc[i, 'Fractions']
                try:
                    txt = 'W' if str(label).lower() == 'waste' else str(int(label))
                except ValueError:
                    txt = str(label)
                ax1.text(x, ymin + tick_h + 5, txt, color='black', fontsize=font_frac,
                         ha='center', va='bottom', rotation=0, clip_on=False)
                last_label_x = x

    ax2 = None
    if y2_label and y2_label in df.columns:
        unit = ""
        try:
            unit = f" ({data[y2_label]['unit']})" if 'unit' in data[y2_label] else ""
        except:
            pass
        label_with_unit = f"{y2_label}{unit}"
        ax2 = ax1.twinx()
        ax2.plot(df["mL"], df[y2_label], label=label_with_unit, color="green")
        ax2.set_ylabel(label_with_unit, color="green", fontsize=font_labels)
        ax2.tick_params(axis='y', labelcolor='green', labelsize=font_ticks)
        ax2.set_ylim(y2_ymin, y2_ymax)

    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels() if ax2 else ([], [])
    ax1.legend(handles1 + handles2, labels1 + labels2, loc='upper right', fontsize=font_legend)
    fig.subplots_adjust(bottom=0.12)
    fig.suptitle(f'Chromatogram – {nom_fitxer}', fontsize=font_title)
    plt.show()

def carregar_i_visualitzar(change=None):
    clear_output(wait=True)
    display(widgets.HTML("<h3>Upload a .zip, .res or .result file:</h3>"))
    display(upload)

    if len(upload.value) == 0:
        print("No file uploaded.")
        return

    nom_fitxer = list(upload.value.values())[0]['metadata']['name']
    fitxer_bytes = list(upload.value.values())[0]['content']
    with open("akta_input", "wb") as f:
        f.write(fitxer_bytes)

    try:
        f = io.StringIO()
        with contextlib.redirect_stdout(f), warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            data = pv.pycorn.load_uni_zip("akta_input")

        available_keys = list(data.keys())
        df = pv.pycorn.utils.get_series_from_data(data, available_keys)

        possibles_y2 = [col for col in available_keys if col not in ["mL", "Fractions", "UV 1_280", "UV 2_260"]]
        y2_dropdown = widgets.Dropdown(options=[""] + possibles_y2, description="Second Y axis")

        uv_max = df[["UV 1_280", "UV 2_260"]].max().max()
        ml_max = df["mL"].max()

        xmin_box, xmin = slider_text_pair("xmin", df["mL"].min(), df["mL"].min(), ml_max)
        xmax_box, xmax = slider_text_pair("xmax", ml_max, df["mL"].min(), ml_max)
        ymin_box, ymin = slider_text_pair("ymin", 0, -100, uv_max + 50)
        ymax_box, ymax = slider_text_pair("ymax", uv_max + 50, 10, uv_max + 300)

        default_y2 = "Conc B" if "Conc B" in df.columns else (possibles_y2[0] if possibles_y2 else "")
        y2_dropdown.value = default_y2 if default_y2 in y2_dropdown.options else ""
        y2_data = df[default_y2] if default_y2 else pd.Series([0])
        y2max_init = y2_data.max() + 10

        y2_ymin_box, y2_ymin = slider_text_pair("Y2 min", 0, -20, y2max_init)
        y2_ymax_box, y2_ymax = slider_text_pair("Y2 max", y2max_init, 10, y2max_init + 50)

        show_fractions = widgets.Checkbox(value=True, description='Show fractions')
        tick_h_box, tick_h = slider_text_pair("Tick height", 10, 1, 50, step=1)
        spacing_box, min_spacing = slider_text_pair("Min spacing", 1.0, 0.1, 10.0, step=0.1)
        font_frac_box, font_frac = slider_text_pair("Fractions font size", 12, 6, 30, step=1)

        font_title_box, font_title = slider_text_pair("Title font size", 18, 10, 30, step=1)
        font_labels_box, font_labels = slider_text_pair("Axis font size", 18, 10, 30, step=1)
        font_ticks_box, font_ticks = slider_text_pair("Ticks font size", 18, 8, 30, step=1)
        font_legend_box, font_legend = slider_text_pair("Legend font size", 18, 8, 30, step=1)

        def on_y2_change(change):
            if change['type'] == 'change' and change['name'] == 'value':
                col = change['new']
                if col and col in df.columns:
                    s = df[col]
                    y2_ymin.value = float(s.min()) - 5
                    y2_ymax.value = float(s.max()) + 10
        y2_dropdown.observe(on_y2_change)

        col1 = widgets.VBox([xmin_box, xmax_box, ymin_box, ymax_box])
        col2 = widgets.VBox([y2_ymin_box, y2_ymax_box, y2_dropdown])
        col3 = widgets.VBox([show_fractions, tick_h_box, spacing_box, font_frac_box])
        col4 = widgets.VBox([font_title_box, font_labels_box, font_ticks_box, font_legend_box])
        controls = widgets.HBox([col1, col2, col3, col4])

        def update_plot(**kwargs):
            kwargs['nom_fitxer'] = nom_fitxer
            plot_chromatogram(df, data, **kwargs)

        out = widgets.interactive_output(update_plot, {
            'xmin': xmin,
            'xmax': xmax,
            'ymin': ymin,
            'ymax': ymax,
            'y2_label': y2_dropdown,
            'y2_ymin': y2_ymin,
            'y2_ymax': y2_ymax,
            'show_fractions': show_fractions,
            'tick_h': tick_h,
            'min_spacing': min_spacing,
            'font_frac': font_frac,
            'font_title': font_title,
            'font_labels': font_labels,
            'font_ticks': font_ticks,
            'font_legend': font_legend
        })

        display(widgets.HTML("<hr><h4>Plot parameters:</h4>"))
        display(controls, out)

    except Exception as e:
        print(f"Error loading the file: {e}")

upload.observe(carregar_i_visualitzar, names='value')


HTML(value='<h3>Upload a .zip, .res or .result file:</h3>')

FileUpload(value={'Ni_5ml_3th_trial_27012021 (whitout equilibration) 002.11703.Result': {'metadata': {'name': …

/content/proteovis/proteovis/pycorn/utils.py:31: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.interpolate(method='linear')


HTML(value='<hr><h4>Plot parameters:</h4>')

Output()